In [ ]:
import pymongo
import json

# Establish a connection to MongoDB using PyMongo
client = pymongo.MongoClient('mongodb://mongodb:27017/')
db = client['aggregations']

db.customers.drop()
db.orders.drop()

Let’s consider two collections: `customers` and `orders`.

In [ ]:
customers = [
  { "_id": 1, "name": "Alice" },
  { "_id": 2, "name": "Bob" },
  { "_id": 3, "name": "Charlie" }
]
db.customers.insert_many(customers)

In [ ]:
orders = [
  { "_id": 1, "customer_id": 1, "item": "Book", "quantity": 2 },
  { "_id": 2, "customer_id": 1, "item": "Pen", "quantity": 5 },
  { "_id": 3, "customer_id": 2, "item": "Notebook", "quantity": 1 }
]
db.orders.insert_many(orders)

### Left Outer Join Using `$lookup`

The following aggregation performs a left outer join on the customers and orders collections:

In [ ]:
results = db.customers.aggregate([
  {
    '$lookup': {
      'from': "orders",
      'localField': "_id",
      'foreignField': "customer_id",
      'as': "orders"
    }
  }
])

print(json.dumps(list(results), indent=2))

**Result**

This will return all customers and their corresponding orders. If a customer has no orders, the orders field will be an empty array.

### Simulating Inner Join

To filter out customers without any orders (simulate an inner join), we can use the `$match` stage after the `$lookup`:

In [ ]:
results = db.customers.aggregate([
  {
    '$lookup': {
      'from': "orders",
      'localField': "_id",
      'foreignField': "customer_id",
      'as': "orders"
    }
  },

  {
    '$match': {
      "orders": { '$ne': [] }
    }
  }
])

print(json.dumps(list(results), indent=2))

In [ ]:
results = db.customers.aggregate([
  {
    '$lookup': {
      'from': "orders",
      'localField': "_id",
      'foreignField': "customer_id",
      'as': "orders"
    }
  },

  {
    '$unwind': {
        'path': "$orders",
        'preserveNullAndEmptyArrays': False
    }
  }
])

print(json.dumps(list(results), indent=2))